⚠️ NOTE:
This notebook is for research and validation only.
Production logic lives in the `src/` directory.

In [ ]:
# import sys
# !{sys.executable} -m pip install xgboost


In [1]:
# Project path setup
import sys
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\shubh\crypto-market-opportunity-engine")
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ Project root added:", PROJECT_ROOT)



✅ Project root added: C:\Users\shubh\crypto-market-opportunity-engine


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from src.model import train_model
from src.inference import load_model
import src.config as config



In [3]:
# load processed featured data
DATA_PROCESSED = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_2025_features.parquet"

df = pd.read_parquet(DATA_PROCESSED)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())



Shape: (105092, 26)
Columns: ['open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'log_return', 'volatility', 'rsi', 'rsi_14', 'bb_high', 'bb_low', 'ema_diff', 'vol_ratio', 'ema_diff_lag_1', 'ema_diff_lag_2', 'ema_diff_lag_3', 'rsi_14_lag_1', 'rsi_14_lag_2', 'vol_ratio_lag_1', 'trend', 'target']


In [4]:
# prepare x and y
FEATURE_COLS = config.FEATURE_COLS
TARGET_COL = config.TARGET_COL

X = df[FEATURE_COLS]
y = df[TARGET_COL]

print("X shape:", X.shape)
print("y distribution:")
print(y.value_counts(normalize=True))


X shape: (105092, 10)
y distribution:
target
0    0.501627
1    0.498373
Name: proportion, dtype: float64


In [5]:
# Time-aware train/test split 
split_idx = int(len(df) * 0.8)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (84073, 10)
Test size: (21019, 10)


In [6]:
# Train models
models = ["logreg", "random_forest", "xgboost"]

for model_name in models:
    print(f"\n🚀 Training {model_name}")

    model_path = train_model(
        df=pd.concat([X_train, y_train], axis=1),
        model_name=model_name,
        version="1"
    )

    print(f"✅ Saved model to {model_path}")



🚀 Training logreg
✅ Saved model to C:\Users\shubh\crypto-market-opportunity-engine\models\logreg_v1.pkl

🚀 Training random_forest
✅ Saved model to C:\Users\shubh\crypto-market-opportunity-engine\models\random_forest_v1.pkl

🚀 Training xgboost
✅ Saved model to C:\Users\shubh\crypto-market-opportunity-engine\models\xgboost_v1.pkl


In [7]:
# Evaluate models (ROC-AUC)
for model_name in models:
    model = load_model(model_name, "1")

    preds = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)

    print(f"📈 {model_name} AUC: {auc:.4f}")


📈 logreg AUC: 0.5195
📈 random_forest AUC: 0.5273
📈 xgboost AUC: 0.5197
